<center>
<img src="https://cordoba.tur.ar/wp-content/uploads/2020/04/cropped-logo-Gobierno.png" width="600" height="600" />
 </center>       

<center>
<h3>Secretaría de Transporte de la Provincia de Córdoba<h3>   <h1>Análisis Funcionamiento Boletos de Gratuidad BEG-BOS-BAM-BSC<h1>  <h0>Urbano Capital<h0>
</center>

#### Importación de Librerías a Utilizar

In [1]:
import io
import os
import pandas as pd
import numpy as np
import seaborn as sbn
import xlsxwriter
import openpyxl

%matplotlib inline
import matplotlib.pyplot as plt

#pd.set_option('display.max_columns', None)

#### Lectura Tablas Urbano Capital

En este paso se leen los archivos correspondientes a las tablas de movimientos de usuarios de Córdoba Capital. Se define una ruta del archivo en la primer línea (folder) y se listan los archivos (filelist). El código está escrito para traer todos los archivos dentro de la carpeta que se le indicó al principio. Esto se puede modificar según como nombren a los archivos. Además, en caso de no querer llamar a la lista completa de archivos, se puede definir a mano esa filelist indicando el nombre de los archivos que se quieren leer. **Para identificar las lineas correspondientes a cada programa, se creó una columna nueva 'PROGRAMA' que toma el nombre del archivo como identificador**

In [2]:
"""Se define ruta de los archivos"""
folder = 'C:/Users/Control 2/Documents/ely python/Diciembre/UC/BEG/'

"""Se genera la lista de todos los archivos con las dos opciones, lista entera o manual"""
#filelist = [file for file in os.listdir(folder) if file.endswith('BSC 2022 Mayo Urbano Capital.csv')]
filelist = [file for file in os.listdir(folder)]

dataframe = []

for file in filelist:
    try:
        df = pd.read_csv(folder + str(file), 
                        error_bad_lines=False)
        df['file'] = str(file)
        df['PROGRAMA'] = df['file'].str[0:3]
        df = df.drop(columns='file')
        dataframe.append(df)
    except OSError:
        pass

df = pd.concat(dataframe, axis=0)

C:\Users\Control 2\AppData\Local\Temp\ipykernel_704\2168727867.py:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(folder + str(file),
C:\Users\Control 2\AppData\Local\Temp\ipykernel_704\2168727867.py:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(folder + str(file),


#
t4= threading.Thread(function_name)
t4.start()
def function_name():
   lock.acquire()
   writi = ExcelWriter('./Final.xlsx')                                                               
   stock = glob.glob("./*.csv")                                                                                           
   df_file = (pd.read_csv(g) for g in stock)
  for inn, di in enumerate(df_file):
     di.to_excel(writi, sheet_name='view{}.csv'.format(inn)
  writi.save()
  lock.release()

In [3]:
df.columns

Index(['ID_AUTORIZACION', 'CUIL', 'ID_ORIGEN', 'ID_DESTINO', 'ID_EMPRESA',
       'INICIO_FRANJA_HORARIA', 'FINAL_FRANJA_HORARIA', 'CANT_USO_X_DIA',
       'CANT_USO_X_MES', 'CANT_USO_X_DIA_SEM', 'CANT_USO_X_SEMANA',
       'ID_TIPO_SOLICITANTE', 'VIAJA_FERIADO_SN', 'FEC_DESDE', 'FEC_HASTA',
       'ID_NIVEL_EDUCATIVO', 'EXCEP', 'ID_DISTANCIA', 'ID_MOVIMIENTO',
       'FEC_MOVIMIENTO', 'ID_ORIGEN.1', 'ID_DESTINO.1', 'ID_PROGRAMA',
       'UID_TARJETA', 'LATITUD', 'LONGITUD', 'NRO_INTERNO',
       'ID_ADMINISTRADORA', 'VALIDADO_SN', 'VALORIZAR_SN', 'FEC_LIQUIDACION',
       'LIQUIDADO_SN', 'CUIT_TRANSPORTISTA', 'PRECIO_GOBIERNO',
       'PRECIO_VALORIZADO', 'ID_RESULTADO', 'ID_EMPRESA.1', 'FEC_INGRESO',
       'PROGRAMA'],
      dtype='object')

#### Lectura Tablas Referencias

Además del input principal que es la tabla de movimientos. Para poder trabajar se necesita cargar el archivo que contiene las referencias para las variables codificadas. El procedimiento es el mismo. Se define una ruta para el achivo y luego se lo llama. En este caso, se generaron a partir del mismo archivo, tres tablas distintas cada una conteniendo información de una hoja distinta del archivo excel.

In [4]:
path = 'C:/Users/Control 2/AAI Dropbox/Agustina Ponce/TRANSPORTE/BOLETOS/Python Scripts/Referencias.xlsx'

tipo_solicitante = pd.read_excel(path, sheet_name = 'TIPO_SOLICITANTE', engine='openpyxl')
nivel_educativo = pd.read_excel(path, sheet_name = 'NIVEL_EDUCATIVO', engine='openpyxl')
empresas = pd.read_excel(path, sheet_name = 'EMPRESAS', engine='openpyxl')
regla_bam = pd.read_excel(path, sheet_name = 'REGLA_BAM', engine='openpyxl')

#### Definición Archivo donde se guardarán todos los resultados obtenidos en esta notebook

Antes de comenzar el análisis, creamos un archivo excel vacío donde iremos guardando cada uno de los resultados obtenidos en los siguientes pasos. Acá nuevamente es necesario indicar una ruta y colocarle un nombre al archivo.

In [5]:
writer = pd.ExcelWriter('C:/Users/Control 2/Documents/ely python/Diciembre/MOV_UC_diciembre_2022_BamBOSBSC.xlsx', engine='xlsxwriter')

#### Cambio Formato Fecha

El formato con el que viene la fecha originalmente en la tabla no es conveniente para trabajar por lo que generamos variables nuevas que indiquen el dia, mes y año del movimiento.

In [6]:
base_uc = df
base_uc['DIA_MOVIMIENTO'] = base_uc['FEC_MOVIMIENTO'].str[0:10]
base_uc['ANO_MES_MOVIMIENTO'] = base_uc['FEC_MOVIMIENTO'].str[3:10]
base_uc['ANO_MOVIMIENTO'] = base_uc['FEC_MOVIMIENTO'].str[6:10]
base_uc

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,...,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO
0,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,29/12/2022 02:47:16 p.m.,BEG,01/12/2022,12/2022,2022
1,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,27/12/2022 01:18:38 p.m.,BEG,01/12/2022,12/2022,2022
2,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,27/12/2022 12:52:28 p.m.,BEG,02/12/2022,12/2022,2022
3,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,27/12/2022 12:52:28 p.m.,BEG,02/12/2022,12/2022,2022
4,8480658,27498226839,0.0,0.0,671,04:00,23:59,2.0,0.0,1111100,...,30707788832,NaN,NaN,3.0,671,27/12/2022 01:25:29 p.m.,BEG,06/12/2022,12/2022,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267466,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,29/12/2022 02:40:30 p.m.,BEG,26/12/2022,12/2022,2022
267467,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,29/12/2022 02:40:30 p.m.,BEG,26/12/2022,12/2022,2022
267468,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,29/12/2022 02:52:35 p.m.,BEG,27/12/2022,12/2022,2022
267469,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,02/01/2023 11:36:39 a.m.,BEG,27/12/2022,12/2022,2022


In [7]:
base_uc.shape

(851932, 42)

### Observo si esta completo el mes en todos los datos 

In [8]:
table = pd.pivot_table(base_uc, values ='ID_AUTORIZACION', index =['DIA_MOVIMIENTO'],
                         columns =['PROGRAMA'], aggfunc = 'count')
table

PROGRAMA,BEG
DIA_MOVIMIENTO,
01/12/2022,82067
02/12/2022,75609
03/12/2022,257
04/12/2022,5
05/12/2022,72172
06/12/2022,70213
07/12/2022,65574
08/12/2022,18
10/12/2022,158


##### Veo los movimientos por dia para ver como voy
Recordar los dias que no aparezcan es porque fue feriado, ojo si no es feriado preguntar a Maxi, si es paro tampoco hay

In [9]:
base_uc.groupby(['DIA_MOVIMIENTO', 'PROGRAMA']).agg({'ID_MOVIMIENTO':'count'}).reset_index()

,DIA_MOVIMIENTO,PROGRAMA,ID_MOVIMIENTO
0,01/12/2022,BEG,82067
1,02/12/2022,BEG,75609
2,03/12/2022,BEG,257
3,04/12/2022,BEG,5
4,05/12/2022,BEG,72172
5,06/12/2022,BEG,70213
6,07/12/2022,BEG,65574
7,08/12/2022,BEG,18
8,10/12/2022,BEG,158
9,11/12/2022,BEG,8


## 1° Etapa: Análisis de Comportamiento Usuarios Córdoba Capital

#### Cantidad de Movimientos Totales por Año-Mes y Programa

Para obtener la cantidad de movimientos totales por mes y programa, se agrupa a la tabla por la columna "ANO_MES_MOVIMIENTO" y "PROGRAMA" y se cuenta la cantidad de "ID_MOVIMIENTO". En este caso no es necesario indicar que cuente movimientos únicos, ya que por su naturaleza la columna tiene todos códigos únicos que que representan cada movimiento realizado por los usuarios.

In [10]:
mov_tot = base_uc.groupby(['ANO_MES_MOVIMIENTO', 'PROGRAMA']).agg({'ID_MOVIMIENTO':'count'}).reset_index()
mov_tot.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS'},axis=1)
mov_tot['TIPO_SERVICIO'] = 'Urbano Capital'

Con el código que sigue, generamos una nueva hoja en el archivo excel creado al comienzo donde se va a guardar la tabla "mov_tot". En el parámetro sheet_name se puede elegir el nombre que se le colocará a la hoja del Excel.

In [11]:
mov_tot.to_excel(writer, sheet_name= 'MOV_TOT_MES')

#### Cantidad de Movimientos Totales por Año-Mes por Cat. y Programa

Acá queremos obtener la cantidad de movimientos totales por mes teniendo en cuenta el Programa, el Tipo de Solicitante y el Nivel Educativo al que pertenece (si corresponde). Agrupamos entonces por las variables "ANO_MES_MOVIMIENTO", "PROGRAMA","ID_TIPO_SOLICITANTE" y "ID_NIVEL_EDUCATIVO" y contamos la cantidad de movimientos. Después de obtener el resultado, necesitamos unir esta tabla con la tabla que contiene las referencia de Tipo Solicitante y Nivel Educativo. Para eso, se utiliza la función merge. Como solo el programa BEG cuenta con nivel educativo, necesitamos rellenar esa columna para el resto de los programas ya que la función groupby no muestra las filas con nulls.

In [12]:
mov_cat = base_uc
mov_cat['ID_NIVEL_EDUCATIVO'] = mov_cat['ID_NIVEL_EDUCATIVO'].fillna(0)

In [13]:
mov_cat = base_uc.groupby(['ANO_MES_MOVIMIENTO', 'PROGRAMA','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO']).agg({'ID_MOVIMIENTO':'count'}).reset_index()

mov_cat = mov_cat.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
mov_cat = mov_cat.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

mov_cat = mov_cat.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS_MES'})

mov_cat = mov_cat.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])#.set_index(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_SOLICITANTE','NIVEL_EDUCATIVO'])

#pd.set_option('display.max_rows', None)
mov_cat.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS'},axis=1)
mov_cat['TIPO_SERVICIO'] = 'Urbano Capital'

In [14]:
mov_cat

,ANO_MES_MOVIMIENTO,PROGRAMA,ID_MOVIMIENTO,TIPO_SOLICITANTE,TIPO_PROGRAMA,NIVEL_EDUCATIVO,TIPO_SERVICIO
0,12/2022,BEG,4646,Alumno,BEG,Inicial,Urbano Capital
1,12/2022,BEG,61297,Alumno,BEG,Primario,Urbano Capital
2,12/2022,BEG,357727,Alumno,BEG,Secundario,Urbano Capital
3,12/2022,BEG,46847,Alumno,BEG,Superior,Urbano Capital
4,12/2022,BEG,231731,Alumno,BEG,Universitario,Urbano Capital
5,12/2022,BEG,14494,Docente,BEG,Inicial,Urbano Capital
6,12/2022,BEG,39295,Docente,BEG,Primario,Urbano Capital
7,12/2022,BEG,67862,Docente,BEG,Secundario,Urbano Capital
8,12/2022,BEG,2647,Docente,BEG,Superior,Urbano Capital
9,12/2022,BEG,4090,Docente,BEG,Especial,Urbano Capital


In [15]:
mov_cat.to_excel(writer, sheet_name= 'MOV_CAT_MES')

#### Cantidad de Movimientos por Empresa por Mes y Programa

En este caso, obtenemos la cantidad de movimientos totales por mes teniendo en cuenta la empresa que indica la autorización. Al igual que en el resultado anterior, después de agrupar y obtener el resultado, realizamos un merge con la tabla de referencias para poder obtener el nombre de cada empresa.

In [16]:
mov_emp = base_uc.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','ID_EMPRESA', 'ID_EMPRESA.1'])['ID_MOVIMIENTO'].count().reset_index()
mov_emp = mov_emp.merge(empresas, on='ID_EMPRESA.1', how='left')
mov_emp = mov_emp.rename({'ID_MOVIMIENTO':'CANT_MOVIMIENTOS'}, axis=1)
mov_emp = mov_emp.drop(columns=['ID_TIPO_EMPRESA', 'GRUPO_COMERCIAL', 'TIPO_EMPRESA'])
mov_emp['TIPO_SERVICIO'] = 'Urbano Capital'
mov_emp

,ANO_MES_MOVIMIENTO,PROGRAMA,ID_EMPRESA_x,ID_EMPRESA.1,CANT_MOVIMIENTOS,ID_EMPRESA_y,RAZON_SOC,CUIL,TIPO_SERVICIO
0,12/2022,BEG,501,501,203947,501,CONIFERAL SACIF,30-54662890-2,Urbano Capital
1,12/2022,BEG,501,502,31102,502,TAMSE,30-70802634-0,Urbano Capital
2,12/2022,BEG,501,671,26383,671,ERSA URBANO S.A.,30-70778883-2,Urbano Capital
3,12/2022,BEG,502,501,13350,501,CONIFERAL SACIF,30-54662890-2,Urbano Capital
4,12/2022,BEG,502,502,80056,502,TAMSE,30-70802634-0,Urbano Capital
5,12/2022,BEG,502,671,17692,671,ERSA URBANO S.A.,30-70778883-2,Urbano Capital
6,12/2022,BEG,670,501,1,501,CONIFERAL SACIF,30-54662890-2,Urbano Capital
7,12/2022,BEG,670,502,16,502,TAMSE,30-70802634-0,Urbano Capital
8,12/2022,BEG,670,671,27,671,ERSA URBANO S.A.,30-70778883-2,Urbano Capital
9,12/2022,BEG,671,501,76604,501,CONIFERAL SACIF,30-54662890-2,Urbano Capital


In [17]:
mov_emp.to_excel(writer, sheet_name= 'MOV_EMP_MES')

#### Cantidad Beneficiarios Activos por Año-Mes

Para obtener la cantidad de Beneficiarios Activos por mes, agrupamos teniendo en cuenta CUILs únicos. Como un mismo beneficiario realiza más de un movimiento por mes, necesitamos contar solo una aparición de cada CUIL.

In [18]:
beneficiarios_mes = base_uc.groupby(['ANO_MES_MOVIMIENTO', 'PROGRAMA']).agg({'CUIL':'nunique'}).reset_index()
beneficiarios_mes.rename({'CUIL':'CANT_BENEFICIARIOS'},axis=1)
beneficiarios_mes['TIPO_SERVICIO'] = 'Urbano Capital'

In [19]:
beneficiarios_mes.to_excel(writer, sheet_name= 'BENEFICIARIOS_MES')

#### Cantidad Beneficiarios Activos por Mes por Cat

En este punto, vamos a obtener la cantidad de Beneficiarios Activos por mes teniendo en cuenta el tipo de solicitante y el nivel educativo al que pertenecen. El número obtenido puede no coincidir con el resultado anterior ya que se contarán CUILs únicos dentro de cada categoría y un mismo CUIL puede estar incluido en la categoría Docente y Alumno por ejemplo. El resultado debe ser igual o mayor al obtenido en el paso anterior.

In [20]:
beneficiarios_cat = base_uc.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO']).agg({'CUIL':'nunique'}).reset_index()
beneficiarios_cat = beneficiarios_cat.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
beneficiarios_cat = beneficiarios_cat.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

beneficiarios_cat = beneficiarios_cat.rename({'CUIL':'CANT_BENEFICIARIOS'}, axis=1)
beneficiarios_cat = beneficiarios_cat.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])#.set_index(['ANO_MOVIMIENTO','NIVEL_EDUCATIVO', 'TIPO_SOLICITANTE'])
beneficiarios_cat['TIPO_SERVICIO'] = 'Urbano Capital'

In [21]:
beneficiarios_cat.to_excel(writer, sheet_name= 'BENEFICIARIOS_CAT')

#### Cantidad Beneficios Activos por Año-Mes

En este paso, lo que se busca es obtener la cantidad de Beneficios Activos Totales por Mes. La cantidad de Beneficios y Beneficiarios suele diferir ya que un beneficiario puede obtener más de un beneficio. Se agrupa teniendo en cuenta apariciones únicas de la variable 'ID_AUTORIZACIÓN' que es la que identifica a cada beneficio entregado. 

In [22]:
beneficios_mes = base_uc.groupby(['ANO_MES_MOVIMIENTO', 'PROGRAMA']).agg({'ID_AUTORIZACION':'nunique'}).reset_index()
beneficios_mes.rename({'ID_AUTORIZACION':'CANT_BENEFICIOS'},axis=1)
beneficios_mes['TIPO_SERVICIO'] = 'Urbano Capital'

In [23]:
beneficios_mes.to_excel(writer, sheet_name= 'BENEFICIOS_MES')

#### Cantidad de Beneficios Activos por Programa, Año-Mes y Categoría

Al resultado anterior se lo visualiza acá abierto por Programa, Tipo de Solicitante y Nivel Educativo. En este caso, al tratarse de beneficios y no beneficiarios, el resultado cerrado del punto anterior debe coincidir con el resultado obtenido aquí.

In [24]:
beneficios_cat = base_uc.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO']).agg({'ID_AUTORIZACION':'nunique'})
beneficios_cat = beneficios_cat.reset_index()
beneficios_cat = beneficios_cat.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
beneficios_cat = beneficios_cat.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

beneficios_cat = beneficios_cat.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])#.set_index(['ANO_MOVIMIENTO', 'NIVEL_EDUCATIVO', 'TIPO_SOLICITANTE'])
beneficios_cat.rename({'ID_AUTORIZACION':'CANT_BENEFICIOS'},axis=1)
beneficios_cat['TIPO_SERVICIO'] = 'Urbano Capital'

In [25]:
beneficios_cat.to_excel(writer, sheet_name= 'BENEFICIOS_CAT')

#### Beneficiarios con Más de un Beneficio en Mismo Servicio

En este punto, se busca saber qué cantidad de beneficios tiene cada beneficiario. Para ello, en un primer paso se agrupa a la tabla por CUIL teniendo en cuenta ID_AUTORIZACION únicos. Esto nos mostrará cuantas autorizaciones tiene cada uno de los CUILs que aparece en la tabla. Luego se reagrupa la tabla en función del resultado anterior, contando la cantidad de CUIL únicos por cantidad de autorizaciones.

In [26]:
multibeneficios = base_uc[base_uc['PROGRAMA']=='BEG']
multibeneficios = multibeneficios.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','CUIL']).agg({'ID_AUTORIZACION':'nunique'}).reset_index()
multibeneficios = multibeneficios.rename({'ID_AUTORIZACION':'CANT_AUTORIZACIONES'},axis=1)
multibeneficios = multibeneficios.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','CANT_AUTORIZACIONES']).agg({'CUIL':'count'}).reset_index()
multibeneficios.rename({'CUIL':'BENEFICIARIOS'},axis=1)
multibeneficios['TIPO_SERVICIO'] = 'Urbano Capital'

In [27]:
multibeneficios.to_excel(writer, sheet_name= 'MULTIBENEFICIOS')

### Tasa Utilización por Mes por Autorizacion

La lógica para el cálculo de la Tasa de Utilización por Mes varia según el programa. Este cálculo se hace por separado para cada uno de los 4 programas para finalmente obtener un resultado agrupado

### 1. Tasa Utilización BEG

#### 1.1 Filtra Base para BEG

In [28]:
base_beg = base_uc.copy()
base_beg = base_beg[base_beg['PROGRAMA']=='BEG']

#### 1.2 Calcula Variable Cantidad de Usos Autorizados

En el caso de los boletos BEG Urbano Capital, la variable de interés que nos permite saber los días autorizados para cada beneficiario es "CANT_USO_X_DIA_SEM" por lo que nos interesa analizarla en profundidad. Para empezar, necesitamos que no existan casos nulos en esta columna para poder realizar distintas operaciones. Por eso, rellenamos los valores nulos que puedan aparecer con 0.

In [29]:
base_beg['CANT_USO_X_DIA_SEM'] = base_beg['CANT_USO_X_DIA_SEM'].fillna(0) 

#### 1.2.1 Crea Columna para Cada Día de la Semana con Boletos Autorizados

La variable "CANT_USO_X_DIA_SEM" indica con 1 si es un día en el que beneficiario tiene autorización para utilizar la tarjeta y con 0 en caso contrario. Para saber cuántos días tiene autorizado en total, tenemos que sumar los digitos de esta columna. Para eso, usamos la función digitize que separa cada uno de los dígitos en una columna separada, facilitando el cálculo.

In [30]:
def digit(x, n):
    """Return the n-th digit of integer in base 10"""
    return (x // 10**n) % 10

def digitize(df, key, n):
    """Extract n less significant digits from an integer in base 10"""
    for i in range(n):
        df['x%d' % i] = digit(df[key], n-i-1) 

In [31]:
digitize(base_beg, 'CANT_USO_X_DIA_SEM', 7)

In [32]:
dict = {'x0':'Monday',
        'x1':'Tuesday',
        'x2':'Wednesday',
        'x3':'Thursday',
        'x4':'Friday',
        'x5':'Saturday',
        'x6':'Sunday'
       }
base_beg.rename(columns=dict, inplace=True)

In [33]:
cols_to_sum = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
base_beg['CANT_AUTO_X_SEM'] = base_beg[cols_to_sum].sum(axis=1)
base_beg

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,...,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,CANT_AUTO_X_SEM
0,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,12/2022,2022,1,1,1,1,1,0,0,5
1,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,12/2022,2022,1,1,1,1,1,0,0,5
2,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,12/2022,2022,1,1,1,1,1,0,0,5
3,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,12/2022,2022,1,1,1,1,1,0,0,5
4,8480658,27498226839,0.0,0.0,671,04:00,23:59,2.0,0.0,1111100,...,12/2022,2022,1,1,1,1,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267466,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,12/2022,2022,1,1,0,1,1,0,0,4
267467,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,12/2022,2022,1,1,0,1,1,0,0,4
267468,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,12/2022,2022,1,1,0,1,1,0,0,4
267469,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,12/2022,2022,1,1,0,1,1,0,0,4


#### 1.2.2 Crea Cantidad de Boletos Autorizados por Semana

Con el paso anterior, ya obtuvimos la cantidad de días autorizados por semana. Sin embargo lo que realmente nos interesa, es saber la cantidad de boletos que ese beneficiario puede utilizar por día. Para ello, definimos nuevamente la columna "CANT_AUTO_X_SEM" con la siguiente regla:

1. Si la columna "CANT_USO_X_DIA" es igual a dos, nos está indicando que en cada uno de los días autorizados, el usuario tendrá dos boletos disponibles.
2. Si la columna "CANT_USO_X_DIA" no tiene valor, entonces debemos observar la columna "CANT_USO_X_SEMANA" que directamente nos indicará el total de boletos autorizados por semana.

La regla entonces es, cuando "CANT_USO_X_DIA" = 2, se multiplica la cantidad de dias autorizados por semana (variable creada "CANT_AUTO_X_SEM" por 2) si no, se toma directamente el valor indicado en la columna "CANT_USO_X_SEMANA".

In [34]:
base_beg['CANT_AUTO_X_SEM'] = np.where(base_beg['CANT_USO_X_DIA'] == 2, base_beg['CANT_AUTO_X_SEM']*2, base_beg['CANT_USO_X_SEMANA'])

 #### 1.3 Calcula Tasa Uso BEG

En este punto se obtiene la tasa de utilización mensual de los boletos por Tipo de Solicitante y Nivel Educativo. Para ello se siguen los pasos a continuación:
   1. Se agrupa la tabla por Mes, Tipo de Solicitante, Nivel Educativo, Autorización y Cantidad Autorizada por Semana.
   2. Se modifica el formato de la variable Mes para que quede expresada con el formato de date lo que permitirá seguir trabajando.
   3. Se crea una variable auxiliar "ULT_DIA_MES" que nos indica el ultimo día del mes correspondiente a la variables "ANO_MES_MOVIMIENTO"
   4. Con la función np.busday_count() se obtiene la cantidad de días hábiles en el mes. La función necesita tener indicado un comienzo y un final del intervalo, razón por la cual se creó la variable auxiliar anterior.
   5. Luego de obtener la cantidad de días hábiles en el mes, se divide el resultado por 5 para obtener la cantidad de semanas promedio de ese mes ("CANT_SEM_MES").
   6. Se obtiene la cantidad total de boletos autorizados en el mes multiplicando la cantidad de boletos autorizados por semana por la cantidad de semanas en el mes.
   7. Finalmente, la tasa de uso es obtenida como el cociente entre la cantidad de movimientos utilizados y los autorizados.

In [35]:
tasa_beg = base_beg.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO','ID_AUTORIZACION', 'CANT_AUTO_X_SEM'])['ID_MOVIMIENTO'].count().reset_index()
tasa_beg['ANO_MES_MOVIMIENTO_aux'] = pd.to_datetime(tasa_beg['ANO_MES_MOVIMIENTO'])

tasa_beg['ULT_DIA_MES'] = tasa_beg['ANO_MES_MOVIMIENTO_aux'] + pd.offsets.MonthEnd(1)

tasa_beg['DIAS_HABILES'] = np.busday_count(tasa_beg['ANO_MES_MOVIMIENTO_aux'].values.astype('datetime64[D]'),tasa_beg['ULT_DIA_MES'].values.astype('datetime64[D]'))

tasa_beg['CANT_SEM_MES'] = tasa_beg['DIAS_HABILES']/5

tasa_beg['CANT_AUTO_MES'] = tasa_beg['CANT_SEM_MES']*tasa_beg['CANT_AUTO_X_SEM']
tasa_beg['TASA_USO'] = tasa_beg['ID_MOVIMIENTO']/tasa_beg['CANT_AUTO_MES']

tasa_beg = tasa_beg.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
tasa_beg = tasa_beg.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')
tasa_beg
tasa_beg = tasa_beg.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
tasa_beg = tasa_beg[['ANO_MES_MOVIMIENTO', 'PROGRAMA', 'ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]
tasa_beg


,ANO_MES_MOVIMIENTO,PROGRAMA,ID_AUTORIZACION,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,TASA_USO
0,12/2022,BEG,8294279,Alumno,Inicial,0.113636
1,12/2022,BEG,8295119,Alumno,Inicial,0.045455
2,12/2022,BEG,8306017,Alumno,Inicial,0.500000
3,12/2022,BEG,8308764,Alumno,Inicial,0.250000
4,12/2022,BEG,8310904,Alumno,Inicial,0.045455
...,...,...,...,...,...,...
79464,12/2022,BEG,10258828,Personal de Apoyo,Especial,0.545455
79465,12/2022,BEG,10291696,Personal de Apoyo,Especial,0.500000
79466,12/2022,BEG,10308344,Personal de Apoyo,Especial,0.727273
79467,12/2022,BEG,10433469,Personal de Apoyo,Especial,0.738636


### 2. Tasa Utilización BOS

#### 2.1 Filtra Base para BOS

In [36]:
base_bos = base_uc.copy()
base_bos = base_bos[base_bos['PROGRAMA']=='BOS']

#### inspecciono como es el archivo

In [37]:
base_bos.shape

(0, 42)

In [38]:
base_bos.columns

Index(['ID_AUTORIZACION', 'CUIL', 'ID_ORIGEN', 'ID_DESTINO', 'ID_EMPRESA',
       'INICIO_FRANJA_HORARIA', 'FINAL_FRANJA_HORARIA', 'CANT_USO_X_DIA',
       'CANT_USO_X_MES', 'CANT_USO_X_DIA_SEM', 'CANT_USO_X_SEMANA',
       'ID_TIPO_SOLICITANTE', 'VIAJA_FERIADO_SN', 'FEC_DESDE', 'FEC_HASTA',
       'ID_NIVEL_EDUCATIVO', 'EXCEP', 'ID_DISTANCIA', 'ID_MOVIMIENTO',
       'FEC_MOVIMIENTO', 'ID_ORIGEN.1', 'ID_DESTINO.1', 'ID_PROGRAMA',
       'UID_TARJETA', 'LATITUD', 'LONGITUD', 'NRO_INTERNO',
       'ID_ADMINISTRADORA', 'VALIDADO_SN', 'VALORIZAR_SN', 'FEC_LIQUIDACION',
       'LIQUIDADO_SN', 'CUIT_TRANSPORTISTA', 'PRECIO_GOBIERNO',
       'PRECIO_VALORIZADO', 'ID_RESULTADO', 'ID_EMPRESA.1', 'FEC_INGRESO',
       'PROGRAMA', 'DIA_MOVIMIENTO', 'ANO_MES_MOVIMIENTO', 'ANO_MOVIMIENTO'],
      dtype='object')

In [39]:
pd.value_counts (base_bos['ID_DISTANCIA'])

Series([], Name: ID_DISTANCIA, dtype: int64)

In [40]:
pd.value_counts (base_bos['ID_NIVEL_EDUCATIVO'])

Series([], Name: ID_NIVEL_EDUCATIVO, dtype: int64)

In [41]:
pd.value_counts (base_bos['EXCEP'])

Series([], Name: EXCEP, dtype: int64)

#### Cambio el formato de la fecha

In [42]:
base_bos = df
base_bos['DIA_MOVIMIENTO'] = base_bos['FEC_DESDE'].str[0:10]
base_bos['ANO_MES_MOVIMIENTO'] = base_bos['FEC_DESDE'].str[3:10]
base_bos['ANO_MOVIMIENTO'] = base_bos['FEC_DESDE'].str[6:10]
base_bos

,ID_AUTORIZACION,CUIL,ID_ORIGEN,ID_DESTINO,ID_EMPRESA,INICIO_FRANJA_HORARIA,FINAL_FRANJA_HORARIA,CANT_USO_X_DIA,CANT_USO_X_MES,CANT_USO_X_DIA_SEM,...,CUIT_TRANSPORTISTA,PRECIO_GOBIERNO,PRECIO_VALORIZADO,ID_RESULTADO,ID_EMPRESA.1,FEC_INGRESO,PROGRAMA,DIA_MOVIMIENTO,ANO_MES_MOVIMIENTO,ANO_MOVIMIENTO
0,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,29/12/2022 02:47:16 p.m.,BEG,15/06/2022,06/2022,2022
1,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,27/12/2022 01:18:38 p.m.,BEG,15/06/2022,06/2022,2022
2,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,27/12/2022 12:52:28 p.m.,BEG,15/06/2022,06/2022,2022
3,10531782,23461270069,0.0,0.0,671,00:00,23:59,NaN,NaN,1111100,...,30708026340,"14,3550",NaN,1.0,502,27/12/2022 12:52:28 p.m.,BEG,15/06/2022,06/2022,2022
4,8480658,27498226839,0.0,0.0,671,04:00,23:59,2.0,0.0,1111100,...,30707788832,NaN,NaN,3.0,671,27/12/2022 01:25:29 p.m.,BEG,18/02/2022,02/2022,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267466,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,29/12/2022 02:40:30 p.m.,BEG,11/10/2022,10/2022,2022
267467,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,29/12/2022 02:40:30 p.m.,BEG,11/10/2022,10/2022,2022
267468,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,29/12/2022 02:52:35 p.m.,BEG,11/10/2022,10/2022,2022
267469,11280472,27437122763,0.0,0.0,671,04:00,23:59,2.0,NaN,1101100,...,30707788832,"14,3550",NaN,1.0,671,02/01/2023 11:36:39 a.m.,BEG,11/10/2022,10/2022,2022


In [43]:
base_bos['ANO_MOVIMIENTO'].agg(['min', 'max']) 

min    2022
max    2022
Name: ANO_MOVIMIENTO, dtype: object

#### 2.2 Calcula Variable Cantidad de Usos Autorizados

En el caso de los boletos BOS Urbano Capital, la variable de interés que nos permite saber los días autorizados para cada beneficiario es "CANT_USO_X_DIA_SEM" por lo que nos interesa analizarla en profundidad. Para empezar, necesitamos que no existan casos nulos en esta columna para poder realizar distintas operaciones. Por eso, rellenamos los valores nulos que puedan aparecer con 0.

In [44]:
base_bos['CANT_USO_X_DIA_SEM'] = base_bos['CANT_USO_X_DIA_SEM'].fillna(0) 

#### 2.2.1 Crea Columna para Cada Día de la Semana con Boletos Autorizados

In [45]:
digitize(base_bos, 'CANT_USO_X_DIA_SEM', 7)

In [46]:
dict = {'x0':'Monday',
        'x1':'Tuesday',
        'x2':'Wednesday',
        'x3':'Thursday',
        'x4':'Friday',
        'x5':'Saturday',
        'x6':'Sunday'
       }
base_bos.rename(columns=dict, inplace=True)

In [47]:
cols_to_sum = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
base_bos['CANT_AUTO_X_SEM'] = base_bos[cols_to_sum].sum(axis=1)

#### 2.2.2 Crea Cantidad de Boletos Autorizados por Semana

Con el paso anterior, ya obtuvimos la cantidad de días autorizados por semana. Sin embargo lo que realmente nos interesa, es saber la cantidad de boletos que ese beneficiario puede utilizar por día. Para ello, definimos nuevamente la columna "CANT_AUTO_X_SEM" con la siguiente regla:

A la columna "CANT_USO_X_DIA" se la multiplica por la columna recién creada "CANT_AUTO_X_SEM" para saber la cantidad de boletos total autorizados por semana.

In [48]:
base_bos['CANT_AUTO_X_SEM'] = base_bos['CANT_USO_X_DIA']*base_bos['CANT_AUTO_X_SEM']

 #### 2.3 Calcula Tasa Uso BOS

En este punto se obtiene la tasa de utilización mensual de los boletos por Tipo de Solicitante y Nivel Educativo. Para ello se siguen los pasos a continuación:
   1. Se agrupa la tabla por Mes, Tipo de Solicitante, Autorización y Cantidad Autorizada por Semana.
   2. Se modifica el formato de la variable Mes para que quede expresada con el formato de date lo que permitirá seguir trabajando.
   3. Se crea una variable auxiliar "ULT_DIA_MES" que nos indica el ultimo día del mes correspondiente a la variables "ANO_MES_MOVIMIENTO"
   4. Con la función np.busday_count() se obtiene la cantidad de días hábiles en el mes. La función necesita tener indicado un comienzo y un final del intervalo, razón por la cual se creó la variable auxiliar anterior.
   5. Luego de obtener la cantidad de días hábiles en el mes, se divide el resultado por 5 para obtener la cantidad de semanas promedio de ese mes ("CANT_SEM_MES").
   6. Se obtiene la cantidad total de boletos autorizados en el mes multiplicando la cantidad de boletos autorizados por semana por la cantidad de semanas en el mes.
   7. Finalmente, la tasa de uso es obtenida como el cociente entre la cantidad de movimientos utilizados y los autorizados.

In [49]:
tasa_bos = base_bos.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA', 'ID_TIPO_SOLICITANTE','ID_NIVEL_EDUCATIVO','ID_AUTORIZACION', 'CANT_AUTO_X_SEM'])['ID_MOVIMIENTO'].count().reset_index()
tasa_bos['ANO_MES_MOVIMIENTO_aux'] = pd.to_datetime(tasa_bos['ANO_MES_MOVIMIENTO'])

tasa_bos['ULT_DIA_MES'] = tasa_bos['ANO_MES_MOVIMIENTO_aux'] + pd.offsets.MonthEnd(1)

tasa_bos['DIAS_HABILES'] = np.busday_count(tasa_bos['ANO_MES_MOVIMIENTO_aux'].values.astype('datetime64[D]'),tasa_bos['ULT_DIA_MES'].values.astype('datetime64[D]'))

tasa_bos['CANT_SEM_MES'] = tasa_bos['DIAS_HABILES']/5

tasa_bos['CANT_AUTO_MES'] = tasa_bos['CANT_SEM_MES']*tasa_bos['CANT_AUTO_X_SEM']
tasa_bos['CANT_AUTO_MES'] = np.where(tasa_bos['CANT_AUTO_MES']==0, 16, tasa_bos['CANT_AUTO_MES'])
tasa_bos['TASA_USO'] = tasa_bos['ID_MOVIMIENTO']/tasa_bos['CANT_AUTO_MES']

tasa_bos = tasa_bos.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
tasa_bos = tasa_bos.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

tasa_bos = tasa_bos.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
tasa_bos = tasa_bos[['ANO_MES_MOVIMIENTO', 'PROGRAMA', 'ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]

tasa_bos

,ANO_MES_MOVIMIENTO,PROGRAMA,ID_AUTORIZACION,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,TASA_USO
0,02/2022,BEG,8294279,Alumno,Inicial,0.131579
1,02/2022,BEG,8295119,Alumno,Inicial,0.250000
2,02/2022,BEG,8306017,Alumno,Inicial,0.578947
3,02/2022,BEG,8308764,Alumno,Inicial,0.289474
4,02/2022,BEG,8310904,Alumno,Inicial,0.052632
...,...,...,...,...,...,...
76676,12/2022,BEG,11662340,Alumno,Superior,0.045455
76677,12/2022,BEG,11702839,Alumno,Superior,0.295455
76678,12/2022,BEG,11662346,Docente,Primario,0.363636
76679,12/2022,BEG,11662498,Docente,Secundario,0.227273


### 3. Tasa Utilización BAM

#### 3.1 Filtra Base para BAM

In [50]:
base_bam = base_uc.copy()
base_bam = base_bam[base_bam['PROGRAMA']=='BAM']

#### 3.2 Calcula Tasa Uso BAM

En este punto se obtiene la tasa de utilización mensual de los boletos. Para ello se siguen los pasos a continuación:
   1. Se agrupa la tabla por Mes y Cantidad Autorizada por Mes.
   2. La tasa de uso es obtenida como el cociente entre la cantidad de movimientos utilizados y los autorizados.

In [51]:
tasa_bam = base_bam
tasa_bam = tasa_bam.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA', 'ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO','ID_AUTORIZACION', 'CANT_USO_X_MES'])['ID_MOVIMIENTO'].count().reset_index()
tasa_bam['TASA_USO'] = tasa_bam['ID_MOVIMIENTO']/tasa_bam['CANT_USO_X_MES']

tasa_bam = tasa_bam.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
tasa_bam = tasa_bam.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

tasa_bam = tasa_bam.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
tasa_bam = tasa_bam[['ANO_MES_MOVIMIENTO', 'PROGRAMA', 'ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]

tasa_bam

,ANO_MES_MOVIMIENTO,PROGRAMA,ID_AUTORIZACION,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,TASA_USO


### 4. Tasa Utilización BSC

#### 4.1 Filtra Base para BSC

In [52]:
base_bsc= base_uc.copy()
base_bsc = base_bsc[base_bsc['PROGRAMA']=='BSC']

#### 4.2 Calcula Tasa Uso BSC

En este punto se obtiene la tasa de utilización mensual de los boletos. Para ello se siguen los pasos a continuación:
   1. Se agrupa la tabla por Mes y Cantidad Autorizada por Mes.
   2. La tasa de uso es obtenida como el cociente entre la cantidad de movimientos utilizados y los autorizados.

In [53]:
tasa_bsc = base_bsc
tasa_bsc = tasa_bsc.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA', 'ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO','ID_AUTORIZACION', 'CANT_USO_X_MES'])['ID_MOVIMIENTO'].count().reset_index()
tasa_bsc['TASA_USO'] = tasa_bsc['ID_MOVIMIENTO']/tasa_bsc['CANT_USO_X_MES']

tasa_bsc = tasa_bsc.merge(tipo_solicitante, on='ID_TIPO_SOLICITANTE', how='left')
tasa_bsc = tasa_bsc.merge(nivel_educativo, on='ID_NIVEL_EDUCATIVO', how='left')

tasa_bsc = tasa_bsc.drop(columns = ['ID_TIPO_SOLICITANTE', 'ID_NIVEL_EDUCATIVO'])
tasa_bsc = tasa_bsc[['ANO_MES_MOVIMIENTO', 'PROGRAMA', 'ID_AUTORIZACION', 'TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO', 'TASA_USO']]

tasa_bsc

,ANO_MES_MOVIMIENTO,PROGRAMA,ID_AUTORIZACION,TIPO_SOLICITANTE,NIVEL_EDUCATIVO,TASA_USO


### 5. Tabla Tasa General

In [54]:
tasa = tasa_beg.append([tasa_bam, tasa_bos, tasa_bsc])
tasa['TIPO_SERVICIO'] = 'Urbano Capital'

C:\Users\Control 2\AppData\Local\Temp\ipykernel_704\298157375.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tasa = tasa_beg.append([tasa_bam, tasa_bos, tasa_bsc])


In [55]:
tasa.to_excel(writer, sheet_name= 'TASA_TABLA')

#### Promedio Tasa Uso por Mes

En el punto anterior, se obtuvo la tasa de uso de cada autorización por mes. Aquí se calcula la tasa promedio por Tipo de Solicitante y Nivel Educativo.

In [56]:
tasa_prom = tasa.copy()

tasa_prom = tasa_prom.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','TIPO_SOLICITANTE', 'NIVEL_EDUCATIVO']).agg({'TASA_USO':'mean'})
tasa_prom.reset_index()
tasa_prom['TIPO_SERVICIO'] = 'Urbano Capital'

In [57]:
tasa_prom.to_excel(writer, sheet_name= 'TASA_USO_PROM')

#### Intervalos Tasa de Uso Promedio

Se crean intervalos para la Tasa de Uso, de manera de saber la cantidad de beneficios que cae en cada uno de estos intervalos. La función para crear intervalos es pd.cut() y necesita que sean definidos tanto los límites de cada intervalo (bins) como el nombre (labels).

In [58]:
intervalos = tasa.copy()

bins = [0,0.25, 0.50,0.75, 1.2, 99999]
labels = ['<25%','25-50%','50-75%','75-100%','+100%']

intervalos['INTERVALOS'] = pd.cut(intervalos['TASA_USO'], bins=bins, labels=labels)
intervalos['TIPO_SERVICIO'] = 'Urbano Capital'

In [59]:
intervalos.to_excel(writer, sheet_name= 'INTERVALOS_TABLA')

#### Casos por Intervalos

A partir de la tabla creada en el punto anterior, se agrupan las autorizaciones por intervalo contando la cantidad de casos.

In [60]:
casos_rango = intervalos.copy()
casos_rango = casos_rango.groupby(['ANO_MES_MOVIMIENTO','PROGRAMA','INTERVALOS','TIPO_SOLICITANTE','NIVEL_EDUCATIVO']).agg({'ID_AUTORIZACION':'count'}).reset_index()
casos_rango.reset_index()
casos_rango.rename({'ID_AUTORIZACION':'CANT_CASOS'},axis=1)
casos_rango['TIPO_SERVICIO'] = 'Urbano Capital'

In [61]:
casos_rango.to_excel(writer, sheet_name= 'INTERVALOS_RANGO')

### Guardar Resultados en Excel

Todas las hojas del excel que fuimos definiendo a lo largo de la notebook, las guardamos en este momento como sigue:

In [62]:
writer.save()